In [2]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import os
import acquire_prep as ap
import explore as ez
import modeling as mz

from env import get_connection

import matplotlib.pyplot as plt
import seaborn as sns
from pydataset import data
from math import sqrt
from scipy import stats
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, QuantileTransformer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LassoLars
from sklearn.feature_selection import SelectKBest, RFE, f_regression, SequentialFeatureSelector
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score
from sklearn.feature_selection import f_regression
from sklearn.model_selection import train_test_split

# ACQUIRE

In [3]:
# create helper function to get the necessary connection url.
def get_db_connection(database):
    return get_connection(database)



In [4]:
# connect to sql zillow database

url = "zillow"

    # use this query to get data    
sql_query = '''SELECT *
            FROM predictions_2017
            JOIN properties_2017 on properties_2017.parcelid = predictions_2017.parcelid
            JOIN propertylandusetype on propertylandusetype.propertylandusetypeid = properties_2017.propertylandusetypeid;
            '''

    # assign data to data frame
df = pd.read_sql(sql_query, get_connection(url))

#filter out single family residential as requested
df= df[df['propertylandusedesc'] == 'Single Family Residential']


In [12]:
df = df.dropna()

In [14]:
df.describe()

,id,parcelid,logerror,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bath_rooms,bed_rooms,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyyear,censustractandblock,propertylandusetypeid
count,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# Convert to correct datatype
df['yearbuilt'] = df.yearbuilt.astype(int)

AttributeError: 'DataFrame' object has no attribute 'yearbuilt'

In [11]:
# acquiring, preparing, and splitting the zillow data


    # Drop all rows with NaN values.
df = df.dropna()
    
    # Convert to correct datatype
df['yearbuilt'] = df.yearbuilt.astype(int)
    
    # rename columns
    
df = df.rename(columns={'yearbuilt':'year_built'})
df = df.rename(columns={'bedroomcnt':'bed_rooms'})
df = df.rename(columns={'bathroomcnt':'bath_rooms'})
df = df.rename(columns={'calculatedfinishedsquarefeet':'finished_sqft'})
    

df = df[df.finished_sqft < 15_000]
df = df[df.bed_rooms <= 10]
df = df[df.bath_rooms <= 10]
df = df[df.taxvaluedollarcnt <= df.taxvaluedollarcnt.quantile(0.75)]


    # split
train, validate, test = split_data(df)


NameError: name 'split_data' is not defined

## Acquire.Take away:

Data was acquired from CodeUp DB and 52442 observations and 6 features were used

# PREPARE 

In [16]:
df = acquire_db()

NameError: name 'get_connection' is not defined

In [15]:
# y_validate.head()
plt.figure(figsize=(16,8))
plt.plot(y_validate.G3, y_validate.G3_pred_mean, alpha=.5, color="gray", label='_nolegend_')
plt.annotate("Baseline: Predict Using Mean", (16, 9.5))
plt.plot(y_validate.G3, y_validate.G3, alpha=.5, color="blue", label='_nolegend_')
plt.annotate("The Ideal Line: Predicted = Actual", (.5, 3.5), rotation=15.5)

plt.scatter(y_validate.tax_value, y_validate.G3_pred_lm, 
            alpha=.5, color="red", s=100, label="Model: LinearRegression")
plt.scatter(y_validate.tax_value, y_validate.G3_pred_glm, 
            alpha=.5, color="yellow", s=100, label="Model: TweedieRegressor")
plt.scatter(y_validate.tax_value, y_validate.G3_pred_lm2, 
            alpha=.5, color="green", s=100, label="Model 2nd degree Polynomial")
plt.legend()
plt.xlabel("Actual Final Grade")
plt.ylabel("Predicted Final Grade")
plt.title("Where are predictions more extreme? More modest?")
# plt.annotate("The polynomial model appears to overreact to noise", (2.0, -10))
# plt.annotate("The OLS model (LinearRegression)\n appears to be most consistent", (15.5, 3))
plt.show()


NameError: name 'y_validate' is not defined

<Figure size 1600x800 with 0 Axes>